<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/answer_Generation_Llama3_1_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# stting up

In [1]:
!pip install accelerate
!pip install datasets
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import os
import torch
import json
import pandas as pd
from google.colab import files
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from IPython.display import FileLink, display
from datasets import load_dataset

#**loading the model**

In [3]:
# Load the model and tokenizer
model_id = "meta-llama/Llama-3.1-8B"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create the text generation pipeline
pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer
)

# Test the pipeline
print(pipeline("كم عمرك؟"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'كم عمرك؟ كم عمرك؟ 2006  2.00 $\nمذكرات فتاة ص'}]


# helper functions

In [ ]:
torch.cuda.empty_cache()

In [6]:
def generate_answer(question, num_samples):
    prompt = f" أجب على السؤال التالي بجملة واحدة موجزة ولكن كاملة باللغة العربية.\nQuestion: {question}\nAnswer:"

    # Generate the first 9 answers with temperature=1.0
    outputs_ave_temp = pipeline(
        prompt,
        max_length=100,
        truncation=True,
        temperature=0.5,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=num_samples
    )


    # Combine the results
    responses = [
        output['generated_text'].split("Answer:")[-1].strip() for output in outputs_ave_temp
    ]

    return responses


# main function to to load questions and gerate answers and save them to a file

In [7]:
# Main function
def main():

    # Step 1: Upload and read the file
    file_path = '/content/train-open.json'
    if os.path.exists(file_path):
        print("File found")
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
    else:
      # Load the XQuAD dataset for Arabic (ar)

      mlqa_dataset = load_dataset("mlqa", "mlqa.vi.ar")

      # Load the validation set (the only available split)
      test_set = mlqa_dataset['validation']


      # Create an empty list to store the data
      data = []

      # Loop through  and collect data
      for i in range(len(test_set)):  # Loop through all samples
          context = test_set[i]['context']
          question = test_set[i]['question']
          answer = test_set[i]['answers']


          # Append the data as a dictionary to the list
          data.append({
                  "Context": context,
                  "Question": question,
                  "Answer": answer
            })

    # Convert the list of dictionaries to a pandas DataFram
    df = pd.DataFrame(data)

    # Initialize an empty list to store the results
    results = []

    # Step 2:  Generate answers for a sample of questions
    for index, row in df.sample(n=100).iterrows() :
        question = row['Question']
        original_answer = row.get('Answer', "")  # Use empty string if no original answer provided
        question_id = row.get('question_id', index)  # Use index if no question_id provided

        # Generate 10 answers for the 50 questions
        generated_answers = generate_answer(question=question, num_samples=10)

        # Append the results
        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": original_answer,
            "generated_answers": generated_answers
        })

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    print(results_df)
#change the file name !!!!!!!!!!!!!!!!!!!
    filename = "generated_answers_Llama3.1-mlqa.json"
    # Step 3: Save results to a JSON file
    with open(filename, "w", encoding="utf-8") as json_file:
          json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("Generated answers saved to 'generated_answers.json'")
    files.download(filename)

    #what i think next stpes is
    # Step 4: Clustering
    # Step 5: Calculate probability
    # Step 6: Calculate semantic entropy
    # Step 7: show results

# Run the main function
if __name__ == "__main__":
    main()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_to

    question_id                                           question  \
0           116  بأي نظام عالمي تأثر النظام القضائي الروماني في...   
1            23                  ما اسم الأبن الأكبر لجورج الثالث؟   
2            77                         ما هي أسباب افتقار التصبغ؟   
3             5                ما هي درجات الرمادي في فراء الأسود؟   
4           151               ما هو الشيء الذي قام بيتشاي بدراسته؟   
..          ...                                                ...   
95          114                    يجب إرسال الضوء من أين إلى أين؟   
96          160                                  أين وقعت المذبحة؟   
97           24            ما هي الدورات التي يمكن جورج معرفه بها؟   
98           70             متى تقرر أنه كان ينبغي أن يكون المغني؟   
99          104  ما هي الأغنية المنفردة الثانية التي صدرت ضمن أ...   

                                      original_answer  \
0   {'answer_start': [325], 'text': ['mô hình của ...   
1         {'answer_start': [434], 'text': ['G

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>